In [11]:
import numpy as np
import pandas as pd
import rasters as rt

In [2]:
filename = "BESS/C4_fraction.tif"

In [9]:
df = pd.read_csv("calval_harvard.csv")
lat = df.lat
lon = df.lon

In [80]:
target_df = pd.DataFrame({"lat": lat, "lon": lon})
target_df

,lat,lon
0,42.5378,-72.1715
1,42.5378,-72.1715
2,42.5378,-72.1715
3,42.5393,-72.1779
4,42.5393,-72.1779
...,...,...
8208,42.5369,-72.1727
8209,42.5369,-72.1727
8210,42.5378,-72.1715
8211,42.5393,-72.1779


In [82]:
location_df = target_df.groupby(["lat", "lon"]).first()
location_df

,
lat,lon
42.5369,-72.1727
42.5378,-72.1715
42.5393,-72.1779


In [86]:
location_df.apply(lambda row: rt.Raster.read_point(rt.Point(row.lon, row.lat)), axis=1)

,
lat,lon
42.5369,-72.1727
42.5378,-72.1715
42.5393,-72.1779


In [84]:
location_df["value"] = location_df.apply(lambda row: rt.Raster.read_point(rt.Point(row.lon, row.lat)))
location_df

ValueError: Cannot set a DataFrame with multiple columns to the single column value

In [78]:
def process_group(group):
    group["value"] = rt.Raster.read_point(filename, point=rt.Point(group.iloc[0].lon, group.iloc[0].lat))

    return group

coord_df = pd.DataFrame({"lat": lat, "lon": lon}).groupby(by=["lat", "lon"], group_keys=False).apply(process_group)
coord_df

,lat,lon,value
0,42.5378,-72.1715,0.0
1,42.5378,-72.1715,0.0
2,42.5378,-72.1715,0.0
3,42.5393,-72.1779,0.0
4,42.5393,-72.1779,0.0
...,...,...,...
8208,42.5369,-72.1727,0.0
8209,42.5369,-72.1727,0.0
8210,42.5378,-72.1715,0.0
8211,42.5393,-72.1779,0.0


In [79]:
np.unique(coord_df.value)

array([0.], dtype=float32)

In [92]:
def value_function(lat, lon):
    print(lat, lon)
    return rt.Raster.read_point(filename, rt.Point(lon, lat))

def process_coordinates(latitudes: np.ndarray, longitudes: np.ndarray, value_function):
    df = pd.DataFrame({'latitude': latitudes, 'longitude': longitudes})
    df = df.groupby(['latitude', 'longitude']).apply(lambda x: x.assign(value=value_function(x.name[0], x.name[1])))
    return df

process_coordinates(lat, lon, value_function)

42.5369 -72.1727
42.5378 -72.1715
42.5393 -72.1779


latitude  longitude  value
latitude longitude                                 
42.5369  -72.1727  6      42.5369   -72.1727    0.0
                   7      42.5369   -72.1727    0.0
                   8      42.5369   -72.1727    0.0
                   19     42.5369   -72.1727    0.0
                   20     42.5369   -72.1727    0.0
...                           ...        ...    ...
42.5393  -72.1779  8200   42.5393   -72.1779    0.0
                   8201   42.5393   -72.1779    0.0
                   8206   42.5393   -72.1779    0.0
                   8207   42.5393   -72.1779    0.0
                   8211   42.5393   -72.1779    0.0

[8213 rows x 3 columns]

In [91]:
target_df.groupby([["lat", "lon"]]).apply(lambda x: x.assign(value=rt.Raster.read_point(filename, rt.Point(x.name[1], x.name[0]))))

ValueError: Grouper and axis must be same length

In [99]:
def process_coordinates(latitudes: np.ndarray, longitudes: np.ndarray, filename) -> np.ndarray:
    def value_function(lat, lon, filename):
        return rt.Raster.read_point(filename, rt.Point(lon, lat))
    
    df = pd.DataFrame({'latitude': latitudes, 'longitude': longitudes})
    df = df.groupby(['latitude', 'longitude']).apply(lambda x: x.assign(value=value_function(x.name[0], x.name[1], filename)))
    return np.array(df.value)

process_coordinates(lat, lon, filename)

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [95]:
def process_coordinates(latitudes: np.ndarray, longitudes: np.ndarray, filename: str):
    df = pd.DataFrame({'latitude': latitudes, 'longitude': longitudes})
    df = df.groupby(['latitude', 'longitude']).apply(lambda x: x.assign(value=lambda: rt.Raster.read_point(filename, rt.Point(x.name[1], x.name[0]))))
    return df

process_coordinates(lat, lon, filename)

TypeError: <lambda>() takes 0 positional arguments but 1 was given